In [ ]:
from aiida import load_profile,orm, engine
from datetime import datetime
from dateutil.relativedelta import relativedelta
load_profile();
import ipywidgets as widgets
from IPython.display import display
from widgets import sens_query, inversion_params, import_nc
import aiidalab_widgets_base as awb
from settings import *


out = widgets.Output()

In [ ]:
def make_date_range(start:datetime, end:datetime, months:int, offset:int)->dict:
    dates = {}
    x = start - relativedelta(months=offset)
    end += relativedelta(months=offset)
    while x < end:
        dates[datetime.strftime(x,'%Y-%m-%d')] = datetime.strftime(x+relativedelta(months=months),'%Y-%m-%d')
        x += relativedelta(months=months)
    return dates


def split_chunk(start,end,chunk):

        if chunk =='year':
            return make_date_range(start, end, 12,0)
        elif chunk == '3year':
            return make_date_range(start, end, 12,12)
        elif chunk == 'month':
            return make_date_range(start, end, 1,0)
        else:
            return make_date_range(start, end, 3,1)
        

def codes_list():
    return [
        awb.ComputationalResourcesWidget._full_code_label(c[0])
        for c in orm.QueryBuilder().append(orm.Code).all()
    ]
def default(name):
    list_codes = codes_list()
    return name if name in list_codes else list_codes[0]
   

# Inversion

### Import data

In [ ]:
import_nc = import_nc.Import()
import_nc

### Step 1

In [ ]:
sq = sens_query.SearchSens()
sq

### Step 2

In [ ]:
inv = inversion_params.InversionParams()
inv


### Step 3

In [ ]:
codes = widgets.Dropdown(
            description="code",
            options=codes_list(),
            value = default('inversion@daint')
        )
codes


In [ ]:
prepend_text_ = f'#SBATCH --constrain=mc\n'+\
                f'#SBATCH --account=em05\n'
              

In [ ]:
def prepare_inversion(s_date,e_date):
    
    inverstion_parameters = inv.construct_dict()
    filter_dict = get_dict_from_filter()
    extras_dict = {k:{x.description:x.value for x in v} for k,v in sq.site_extras.items()}

    for i in sq.list_info_obs:
        i.update(extras_dict[i['name']])
        i.update(filter_dict[i['name']])

    inverstion_parameters.update({'sites':{i['name']:i for i in sq.list_info_obs}})

    #other params
    inverstion_parameters.update({'para': sq.species.value,
                                'domain':sq.domain.value,
                                'transport.model':sq.model.value,
                                'dtm.start':s_date,
                                'dtm.end':e_date,
                                'res.dir':'./'})
    
    #test conversion
    inverstion_parameters = {
    key.replace(".", "_"): value for key, value in inverstion_parameters.items()
}
    print(inverstion_parameters)
    remotes_dictionary = {k[:-3].replace('-','_'):v 
                         for k,v in sq.list_remotes.items()}
    
    #builder starts
    builder = INVERSION.get_builder()
    builder.code = orm.load_code(codes.value)
    builder.remotes = remotes_dictionary
    builder.observations = sq.selected_obs
    builder.metadata.options.custom_scheduler_commands = prepend_text_
    builder.inv_params = orm.Dict(inverstion_parameters)
    builder.start_date = orm.Str(s_date)
    builder.end_date = orm.Str(e_date)
    builder.chunk = orm.Str(inv.chunk.value)
    builder.chunk_w = orm.Str(inv.chunk_w.value)

    """builder.metadata.options.stash = {
        'source_list': ['aiida.out','*'],
        'target_base': stash_address + f'{username}/aiida_stash',
        'stash_mode': StashMode.COPY.value,
    }"""

    return builder

def get_dict_from_filter():
    filter_dict  ={}
    for k,v in sq.site_filter.items():
        filter_={}
        for i in v.children[0].children[0].children:
            f = {}
            for j in range(4):
                f[i.children[j].description] = i.children[j].value
            filter_[i.children[0].value]=f
        filter_dict[k] = filter_
    return filter_dict
        

def run_inversions():

    text = """Submitted inversion from {0} to {1}. Click
        <a href=/apps/apps/aiidalab-widgets-base/notebooks/process.ipynb?id={2}
        target="_blank">here</a> to follow.<br>"""
    submision_text = widgets.HTML('')
    start = datetime.strptime(sq.date_range.value[:10], '%Y-%m-%d')
    end = datetime.strptime(sq.date_range.value[12:], '%Y-%m-%d')
    dates = split_chunk(start, end, inv.chunk_w.value)

    for s,e in dates.items():
        process = engine.submit(prepare_inversion(s,e))
        submision_text.value += text.format(s,e,process.pk)

    display(submision_text)


In [ ]:
button_inversion = widgets.Button(
    description='Submit inversion',
)
def on_click(b):   
    with out:
        run_inversions()
       
button_inversion.on_click(on_click)
button_inversion